Drag all the ETFs data from yahoo finance

In [1]:
import pandas as pd
import yfinance
from datetime import datetime

In [2]:
import os
print(os.getcwd())

/Users/fushengluo/Desktop/Johns Hopkins Graduate/Modules/MachineLearninginFinance/Project/Dataset/S&P500_stocks


In [3]:
startDate = datetime(2024,3,11)
endDate = datetime(2025,4,25)

In [10]:
# Use all tickers for ETFs
tickers_table = pd.read_csv("S&P500_tickers.csv", header=None)

In [14]:
tickers_table.columns = ['Ticker']
tickers_table.head()


Ticker
0    MMM
1    AOS
2    ABT
3   ABBV
4    ACN

In [15]:
my_tickers = tickers_table["Ticker"]

In [16]:
my_tickers = my_tickers.tolist()

In [17]:
print(my_tickers[:10])
print(f"There are {len(my_tickers)} ETFs")

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']
There are 503 ETFs


In [18]:
yahooDataFrame = yfinance.download(my_tickers,startDate,endDate)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  503 of 503 completed

503 Failed downloads:
['FICO', 'NWS', 'CZR', 'PCG', 'TROW', 'GDDY', 'ALLE', 'JBL', 'PLTR', 'EXPD', 'ZTS', 'MGM', 'AMT', 'GIS', 'STE', 'OKE', 'ADM', 'STZ', 'NTAP', 'MCD', 'MA', 'CBRE', 'LVS', 'CARR', 'ERIE', 'PEP', 'AIZ', 'MNST', 'NTRS', 'MKTX', 'IEX', 'INTC', 'MTB', 'SBAC', 'DD', 'AIG', 'UNH', 'FAST', 'EQIX', 'TJX', 'DFS', 'WBA', 'AEE', 'HWM', 'NOW', 'DRI', 'GILD', 'CPT', 'KHC', 'LHX', 'BAX', 'IT', 'CAG', 'KKR', 'EW', 'DG', 'PAYX', 'MSCI', 'BRK.B', 'EXE', 'MCHP', 'ANET', 'WAT', 'INTU', 'ROK', 'SWKS', 'GS', 'DOC', 'KEY', 'OMC', 'ISRG', 'NWSA', 'SYY', 'MKC', 'CFG', 'HPE', 'BG', 'VLO', 'ETN', 'C', 'BBY', 'GWW', 'BMY', 'APA', 'GD', 'DXCM', 'NFLX', 'VLTO', 'RVTY', 'CL', 'SYK', 'ECL', 'EPAM', 'PPG', 'AXON', 'LUV', 'UHS', 'QCOM', 'NUE', 'MCK', 'MCO', 'LLY', 'VZ', 'BALL', 'PSA', 'EMR', 'LW', 'BEN', 'VST', 'DELL', 'TDG', 'ACN', 'HAL', 'ZBH', 'LII', 'TXN', 'DHR', 'NRG', 'PG', 'MAS', 'PSX', 'LEN', 'ANSS', 'FTNT', 'AFL', 'HO

In [40]:
# Read the CSV, skipping the first 2 rows which contain partial headers
# The real data starts at row 3 (index 2) with dates
df = pd.read_csv('S&P500_prices.csv', skiprows=2)

# Clean up column names - they're currently in row 0 (Ticker) and row 1 (Price types)
# We'll combine them to create proper multi-level column names
headers = pd.read_csv('S&P500_prices.csv', header=None, nrows=2)
column_names = [f"{price_type}_{ticker}" if pd.notna(ticker) else price_type 
               for price_type, ticker in zip(headers.iloc[0], headers.iloc[1])]

# Apply the new column names
df.columns = column_names

# Clean up the Date column (which may have NaN in header)
df = df.rename(columns={df.columns[0]: 'Date'})

# Remove any rows where Date is NaN (like the original NaN row)
df = df.dropna(subset=['Date'])

# Convert Date to datetime and set as index
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

# Drop any entirely empty columns
df = df.dropna(axis=1, how='all')

print(df.head())

               Close_A  Close_AAPL  Close_ABBV  Close_ABNB   Close_ABT  \
Date                                                                     
2024-03-11  145.964951  171.938141  171.729462  162.990005  117.246452   
2024-03-12  146.510010  172.415863  172.962708  166.669998  117.802483   
2024-03-13  146.490173  170.325760  171.949326  164.759995  117.217178   
2024-03-14  143.903641  172.186951  173.230408  166.440002  115.939262   
2024-03-15  146.153229  171.808746  170.056427  160.639999  112.661545   

            Close_ACGL   Close_ACN  Close_ADBE   Close_ADI  Close_ADM  ...  \
Date                                                                   ...   
2024-03-11   85.875641  365.285065  560.419983  194.973846  52.906952  ...   
2024-03-12   86.455696  372.390747  579.140015  195.947662  54.988155  ...   
2024-03-13   86.531769  370.668152  573.549988  195.357483  56.144382  ...   
2024-03-14   86.560295  370.169006  570.450012  191.255554  56.086571  ...   
2024-03-15   

Transform into a long format

In [51]:
df_long = df.reset_index()
df_long.columns

Index(['Date', 'Close_A', 'Close_AAPL', 'Close_ABBV', 'Close_ABNB',
       'Close_ABT', 'Close_ACGL', 'Close_ACN', 'Close_ADBE', 'Close_ADI',
       ...
       'Volume_WTW', 'Volume_WY', 'Volume_WYNN', 'Volume_XEL', 'Volume_XOM',
       'Volume_XYL', 'Volume_YUM', 'Volume_ZBH', 'Volume_ZBRA', 'Volume_ZTS'],
      dtype='object', length=2506)

In [54]:
# 1. First ensure Date is a column (not index)
if 'Date' not in df.columns:
    df = df.reset_index()  # Moves index to column if Date was the index

# 2. Melt to long format - verify column exists first
if 'Date' in df.columns:
    df_long = df.melt(
        id_vars=['Date'],
        var_name='Feature_Ticker', 
        value_name='Value'
    )
else:
    raise ValueError("No 'Date' column found in DataFrame")

# 3. Extract Ticker and Feature using improved regex
extractions = df_long['Feature_Ticker'].str.extract(
    r'^(Open|High|Low|Close|Volume)_([A-Za-z\.]+)$'
)
df_long[['Feature', 'Ticker']] = extractions

# 4. Clean up - drop invalid rows
df_long = df_long.dropna(subset=['Feature', 'Ticker'])

# 5. Pivot to final format
df_final = df_long.pivot_table(
    index=['Date', 'Ticker'],
    columns='Feature',
    values='Value',
    aggfunc='first'
).reset_index()

# 6. Ensure all required columns exist
required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
for col in required_columns:
    if col not in df_final.columns:
        df_final[col] = None  # Add missing columns with null values

# 7. Select and order columns
df_final = df_final[['Date', 'Ticker'] + required_columns]

# 8. Save to CSV
df_final.to_csv('stock_data_final.csv', index=False)

print("Successfully transformed data:")
print(df_final.head())

Successfully transformed data:
Feature       Date Ticker        Open        High         Low       Close  \
0       2024-03-11      A  146.668570  147.937054  144.805478  145.964951   
1       2024-03-11   AAPL  172.127250  173.560485  171.241434  171.938141   
2       2024-03-11   ABBV  170.563119  173.287761  170.400598  171.729462   
3       2024-03-11   ABNB  163.759995  164.259995  161.975006  162.990005   
4       2024-03-11    ABT  117.968327  118.446324  116.690412  117.246452   

Feature      Volume  
0         1397600.0  
1        60139500.0  
2         5134800.0  
3         3149600.0  
4         3886000.0  


Store in the csv file

In [56]:
df_final.to_csv('S&P500_longformat_price.csv')

## Delete the redundant tickers

In [1]:
import numpy as np
import pandas as pd

In [2]:
to_remove_list = ['APD',
 'DFS',
 'BAX',
 'GE',
 'CMI',
 'BDX',
 'BSX',
 'GIS',
 'A',
 'BRO',
 'FE',
 'AMP',
 'EG',
 'AWK',
 'CFG',
 'EXR',
 'COF',
 'BLK',
 'APH',
 'COR',
 'DUK',
 'BR',
 'HSY',
 'BA',
 'FDX',
 'DVN',
 'FCX',
 'HII',
 'AJG',
 'MO',
 'CMS',
 'ADM',
 'FRT',
 'AEE',
 'DG',
 'C',
 'F',
 'STZ',
 'ATO',
 'ELV',
 'ETR',
 'CNC',
 'T',
 'HPQ',
 'AXP',
 'FTV',
 'HPE',
 'CAG',
 'HRL',
 'ED',
 'DLR',
 'CCL',
 'ECL',
 'HLT',
 'HD',
 'CAT',
 'CVX',
 'IBM',
 'DAL',
 'DHR',
 'HAL',
 'BBY',
 'BAC',
 'ACN',
 'GS',
 'KO',
 'ABT',
 'GEV',
 'GM',
 'AVY',
 'XOM',
 'BK',
 'HIG',
 'BX',
 'CMG',
 'HWM',
 'IEX',
 'IFF',
 'CRL',
 'AMT',
 'BEN',
 'EOG',
 'EMN',
 'ETN',
 'CLX',
 'ALB',
 'BMY',
 'DOV',
 'DD',
 'COP',
 'IR',
 'HCA',
 'CL',
 'CVS',
 'DE',
 'FI',
 'ALL',
 'CCI',
 'EMR',
 'CB',
 'EIX',
 'GLW',
 'D',
 'HES',
 'EL',
 'AOS',
 'CI',
 'AZO',
 'GPC',
 'DAY',
 'BXP',
 'IT',
 'EFX',
 'DHI',
 'CAH',
 'ITW',
 'AFL',
 'AVB',
 'AON',
 'GPN',
 'EQT',
 'FDS',
 'EW',
 'HUM',
 'GD',
 'CHD',
 'CNP',
 'EQR',
 'ES',
 'BG',
 'DTE',
 'GL',
 'KMX',
 'ARE',
 'AES',
 'DRI',
 'ESS',
 'AME',
 'IP',
 'CPT',
 'FIS',
 'AIG',
 'DVA',
 'AIZ',
 'DOW',
 'DOC',
 'APO',
 'CF',
 'ICE',
 'TECH',
 'IPG',
 'IVZ',
 'IQV',
 'IRM',
 'JBL',
 'J',
 'JNJ',
 'JCI',
 'JPM',
 'K',
 'KEY',
 'KMB',
 'KIM',
 'KMI',
 'KKR',
 'KR',
 'LHX',
 'LH',
 'LW',
 'LVS',
 'LEN',
 'LII',
 'LLY',
 'LYV',
 'LMT',
 'L',
 'LOW',
 'LYB',
 'MTB',
 'MPC',
 'MMC',
 'MLM',
 'MAS',
 'MA',
 'MKC',
 'MCD',
 'MCK',
 'MDT',
 'MRK',
 'MET',
 'MTD',
 'MGM',
 'MAA',
 'MHK',
 'MOH',
 'TAP',
 'MCO',
 'MS',
 'MOS',
 'MSI',
 'NEM',
 'NEE',
 'NKE',
 'NI',
 'NSC',
 'NOC',
 'NRG',
 'NUE',
 'NVR',
 'OXY',
 'OMC',
 'OKE',
 'PKG',
 'PH',
 'PNR',
 'PFE',
 'PCG',
 'PM',
 'PSX',
 'PNW',
 'PNC',
 'PPG',
 'PPL',
 'PG',
 'PGR',
 'PLD',
 'PRU',
 'PEG',
 'PSA',
 'PHM',
 'PWR',
 'DGX',
 'RL',
 'RJF',
 'RTX',
 'O',
 'RF',
 'RSG',
 'RMD',
 'ROK',
 'ROL',
 'RCL',
 'CRM',
 'SLB',
 'STX',
 'SRE',
 'NOW',
 'SHW',
 'SPG',
 'SJM',
 'SW',
 'SNA',
 'SO',
 'LUV',
 'SWK',
 'STT',
 'STE',
 'SYK',
 'SYF',
 'SYY',
 'TPR',
 'TGT',
 'TEL',
 'TDY',
 'TPL',
 'TXT',
 'TMO',
 'TJX',
 'TKO',
 'TT',
 'TDG',
 'TRV',
 'TFC',
 'TYL',
 'TSN',
 'USB',
 'UDR',
 'UNP',
 'UPS',
 'URI',
 'UNH',
 'UHS',
 'VLO',
 'VTR',
 'VZ',
 'V',
 'VST',
 'VMC',
 'WRB',
 'GWW',
 'WAB',
 'WMT',
 'DIS',
 'WM',
 'WAT',
 'WEC',
 'WFC',
 'WST',
 'WY',
 'WSM',
 'WMB',
 'XYL',
 'YUM',
 'ZBH',
 'ZTS']

In [3]:
df = pd.read_csv("S&P500_longformat_price.csv",index_col='Date')

In [4]:
df.head()

Unnamed: 0 Ticker        Open        High         Low       Close  \
Date                                                                            
2024-03-11           0      A  146.668570  147.937054  144.805478  145.964951   
2024-03-11           1   AAPL  172.127250  173.560485  171.241434  171.938141   
2024-03-11           2   ABBV  170.563119  173.287761  170.400598  171.729462   
2024-03-11           3   ABNB  163.759995  164.259995  161.975006  162.990005   
2024-03-11           4    ABT  117.968327  118.446324  116.690412  117.246452   

                Volume  
Date                    
2024-03-11   1397600.0  
2024-03-11  60139500.0  
2024-03-11   5134800.0  
2024-03-11   3149600.0  
2024-03-11   3886000.0

In [7]:
df_filtered = df[df['Ticker'].isin(to_remove_list)]
df_filtered.head()

Unnamed: 0 Ticker        Open        High         Low       Close  \
Date                                                                            
2024-03-11           0      A  146.668570  147.937054  144.805478  145.964951   
2024-03-11           4    ABT  117.968327  118.446324  116.690412  117.246452   
2024-03-11           6    ACN  366.420406  367.575311  359.265824  365.285065   
2024-03-11           9    ADM   52.694976   53.533242   52.675705   52.906952   
2024-03-11          12    AEE   70.142268   70.919704   70.113475   70.689354   

               Volume  
Date                   
2024-03-11  1397600.0  
2024-03-11  3886000.0  
2024-03-11  2220000.0  
2024-03-11  5209600.0  
2024-03-11  1578300.0

In [8]:
remain_list = np.unique(df_filtered['Ticker'])
print(len(remain_list))

305


In [10]:
df_filtered = df_filtered.drop(columns="Unnamed: 0")

In [11]:
df_filtered.head()

Ticker        Open        High         Low       Close     Volume
Date                                                                        
2024-03-11      A  146.668570  147.937054  144.805478  145.964951  1397600.0
2024-03-11    ABT  117.968327  118.446324  116.690412  117.246452  3886000.0
2024-03-11    ACN  366.420406  367.575311  359.265824  365.285065  2220000.0
2024-03-11    ADM   52.694976   53.533242   52.675705   52.906952  5209600.0
2024-03-11    AEE   70.142268   70.919704   70.113475   70.689354  1578300.0

In [12]:
df_filtered.to_csv("S&P305_stocks.csv")

         Date Ticker        Open        High         Low       Close  \
0  2024-03-11      A  146.668570  147.937054  144.805478  145.964951   
1  2024-03-11    ABT  117.968327  118.446324  116.690412  117.246452   
2  2024-03-11    ACN  366.420406  367.575311  359.265824  365.285065   
3  2024-03-11    ADM   52.694976   53.533242   52.675705   52.906952   
4  2024-03-11    AEE   70.142268   70.919704   70.113475   70.689354   

      Volume  
0  1397600.0  
1  3886000.0  
2  2220000.0  
3  5209600.0  
4  1578300.0  
(85915, 7)


## Combine Sentiments with stock prices

In [1]:
import numpy as np
import pandas as pd

Stock price

In [2]:
df_stock = pd.read_csv("S&P305_stocks.csv")
print(df_stock.head())
print(df_stock.shape)

         Date Ticker        Open        High         Low       Close  \
0  2024-03-11      A  146.668570  147.937054  144.805478  145.964951   
1  2024-03-11    ABT  117.968327  118.446324  116.690412  117.246452   
2  2024-03-11    ACN  366.420406  367.575311  359.265824  365.285065   
3  2024-03-11    ADM   52.694976   53.533242   52.675705   52.906952   
4  2024-03-11    AEE   70.142268   70.919704   70.113475   70.689354   

      Volume  
0  1397600.0  
1  3886000.0  
2  2220000.0  
3  5209600.0  
4  1578300.0  
(85915, 7)


### Our model

In [15]:
df_sent = pd.read_csv("S&P500_sentiments.csv")
print(df_sent.head())
print(df_sent.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'S&P500_sentiments.csv'

In [ ]:
# Step 1: Ensure both key columns have the same format
df_stock['Date'] = pd.to_datetime(df_stock['Date'])
df_sent['date'] = pd.to_datetime(df_sent['date'])

# Step 2: Standardize column names
df_sent.rename(columns={'date': 'Date', 'ticker': 'Ticker'}, inplace=True)

# Step 3: Merge on 'Date' and 'Ticker'
df_merged = pd.merge(df_stock, df_sent, on=['Date', 'Ticker'], how='left')


In [ ]:
df_merged.head()

Date Ticker        Open        High         Low       Close  \
0 2024-03-11      A  146.668570  147.937054  144.805478  145.964951   
1 2024-03-11    ABT  117.968327  118.446324  116.690412  117.246452   
2 2024-03-11    ACN  366.420406  367.575311  359.265824  365.285065   
3 2024-03-11    ADM   52.694976   53.533242   52.675705   52.906952   
4 2024-03-11    AEE   70.142268   70.919704   70.113475   70.689354   

      Volume  sentiment  
0  1397600.0        NaN  
1  3886000.0        NaN  
2  2220000.0        NaN  
3  5209600.0        NaN  
4  1578300.0        NaN

In [ ]:
df_merged.to_csv("stocks_signal_ourmodel.csv")

In [ ]:
np.unique(df_merged['sentiment'])

array([-1.        , -0.5       , -0.33333333, -0.25      , -0.22222222,
       -0.2       , -0.16666667, -0.14285714, -0.125     , -0.11111111,
        0.        ,  0.11111111,  0.125     ,  0.14285714,  0.16666667,
        0.2       ,  0.22222222,  0.25      ,  0.28571429,  0.33333333,
        0.4       ,  0.42857143,  0.5       ,  0.66666667,  0.75      ,
        1.        ,         nan])

### ProsusAI_bert

In [10]:
df_sent = pd.read_csv('NewsSentiment_ProsusAI_bert.csv')
df_sent.head()

date ticker  sentiment
0  2024-03-11    GEV   1.000000
1  2024-03-12    GEV  -0.333333
2  2024-03-13    GEV   1.000000
3  2024-03-14    GEV   1.000000
4  2024-03-15    GEV  -1.000000

In [11]:
# Step 1: Ensure both key columns have the same format
df_stock['Date'] = pd.to_datetime(df_stock['Date'])
df_sent['date'] = pd.to_datetime(df_sent['date'])

# Step 2: Standardize column names
df_sent.rename(columns={'date': 'Date', 'ticker': 'Ticker'}, inplace=True)

# Step 3: Merge on 'Date' and 'Ticker'
df_merged = pd.merge(df_stock, df_sent, on=['Date', 'Ticker'], how='left')


In [12]:
df_merged.set_index('Date')
df_merged.head()

Date Ticker        Open        High         Low       Close  \
0 2024-03-11      A  146.668570  147.937054  144.805478  145.964951   
1 2024-03-11    ABT  117.968327  118.446324  116.690412  117.246452   
2 2024-03-11    ACN  366.420406  367.575311  359.265824  365.285065   
3 2024-03-11    ADM   52.694976   53.533242   52.675705   52.906952   
4 2024-03-11    AEE   70.142268   70.919704   70.113475   70.689354   

      Volume  sentiment  
0  1397600.0        NaN  
1  3886000.0        NaN  
2  2220000.0        NaN  
3  5209600.0        NaN  
4  1578300.0        NaN

In [13]:
df_merged.to_csv('stocks_signal_ProsusAI_bert.csv')

### NewsSentiment_ahmedrachid/FinancialBERT

In [17]:
df_sent = pd.read_csv('NewsSentiment_ahmedrachid_FinancialBERT.csv')
df_sent.head()

Unnamed: 0        Date Ticker        Open        High         Low  \
0           0  2024-03-11      A  146.668570  147.937054  144.805478   
1           1  2024-03-11    ABT  117.968327  118.446324  116.690412   
2           2  2024-03-11    ACN  366.420406  367.575311  359.265824   
3           3  2024-03-11    ADM   52.694976   53.533242   52.675705   
4           4  2024-03-11    AEE   70.142268   70.919704   70.113475   

        Close     Volume  sentiment  
0  145.964951  1397600.0        NaN  
1  117.246452  3886000.0        NaN  
2  365.285065  2220000.0        NaN  
3   52.906952  5209600.0        NaN  
4   70.689354  1578300.0        NaN

In [19]:
dupes = df_sent[df_sent.duplicated(subset=['Date', 'Ticker'], keep=False)]
print(dupes)

       Unnamed: 0        Date Ticker        Open        High         Low  \
2537         2537  2024-03-21    FRT   97.484265   98.280409   97.148544   
2538         2538  2024-03-21    FRT   97.484265   98.280409   97.148544   
2728         2728  2024-03-22      A  147.510919  147.937043  145.102772   
2729         2729  2024-03-22      A  147.510919  147.937043  145.102772   
2733         2733  2024-03-22    AEE   70.195358   70.195358   69.265491   
...           ...         ...    ...         ...         ...         ...   
97320       97320  2025-04-23    APD  269.700012  273.589996  262.920013   
97455       97455  2025-04-23    JBL  139.820007  143.380005  138.399994   
97456       97456  2025-04-23    JBL  139.820007  143.380005  138.399994   
97469       97469  2025-04-23      L   86.279999   86.790001   85.540001   
97470       97470  2025-04-23      L   86.279999   86.790001   85.540001   

            Close     Volume  sentiment  
2537    98.050201   845100.0        1.0  
253

In [21]:
df_grouped = df_sent.groupby(['Date', 'Ticker'])['sentiment'].mean().reset_index()
df_grouped.set_index('Date', inplace=True)

In [24]:
np.unique(df_grouped['sentiment'])

array([-1.        , -0.85714286, -0.83333333, -0.8       , -0.75      ,
       -0.71428571, -0.66666667, -0.625     , -0.6       , -0.57142857,
       -0.55555556, -0.5       , -0.42857143, -0.4       , -0.375     ,
       -0.33333333, -0.28571429, -0.25      , -0.2       , -0.16666667,
       -0.14285714, -0.125     , -0.11111111,  0.        ,  0.1       ,
        0.11111111,  0.125     ,  0.14285714,  0.16666667,  0.2       ,
        0.22222222,  0.25      ,  0.28571429,  0.33333333,  0.375     ,
        0.4       ,  0.42857143,  0.5       ,  0.57142857,  0.6       ,
        0.625     ,  0.66666667,  0.71428571,  0.75      ,  0.8       ,
        0.83333333,  0.85714286,  1.        ,         nan])

In [46]:
df_sent = df_grouped.copy()

In [47]:
df_sent = df_sent.rename({'Date':'date'})

In [48]:
df_sent = df_sent.reset_index()
df_sent = df_sent.rename(columns={'Date':'date','Ticker':'ticker'})

In [49]:
df_sent.head()

date ticker  sentiment
0  2024-03-11      A        NaN
1  2024-03-11    ABT        NaN
2  2024-03-11    ACN        NaN
3  2024-03-11    ADM        NaN
4  2024-03-11    AEE        NaN

In [50]:
# Step 1: Ensure both key columns have the same format
df_stock['Date'] = pd.to_datetime(df_stock['Date'])
df_sent['date'] = pd.to_datetime(df_sent['date'])

# Step 2: Standardize column names
df_sent.rename(columns={'date': 'Date', 'ticker': 'Ticker'}, inplace=True)

# Step 3: Merge on 'Date' and 'Ticker'
df_merged = pd.merge(df_stock, df_sent, on=['Date', 'Ticker'], how='left')


In [52]:
df_merged.set_index('Date')
#df_merged = df_merged.drop(columns='title')
df_merged.head()

Date Ticker        Open        High         Low       Close  \
0 2024-03-11      A  146.668570  147.937054  144.805478  145.964951   
1 2024-03-11    ABT  117.968327  118.446324  116.690412  117.246452   
2 2024-03-11    ACN  366.420406  367.575311  359.265824  365.285065   
3 2024-03-11    ADM   52.694976   53.533242   52.675705   52.906952   
4 2024-03-11    AEE   70.142268   70.919704   70.113475   70.689354   

      Volume  sentiment  
0  1397600.0        NaN  
1  3886000.0        NaN  
2  2220000.0        NaN  
3  5209600.0        NaN  
4  1578300.0        NaN

In [53]:
df_merged.to_csv('stocks_signal_ahmedrachid_FinancialBERT.csv')

### NewsSentiment_yiyanghkust

In [55]:
df_sent = pd.read_csv('NewsSentiment_yiyanghkust.csv')
df_sent.head()

date ticker  sentiment
0  2024-03-11    GEV  -1.000000
1  2024-03-12    GEV  -0.333333
2  2024-03-13    GEV  -1.000000
3  2024-03-14    GEV  -1.000000
4  2024-03-15    GEV   0.000000

In [56]:
# Step 1: Ensure both key columns have the same format
df_stock['Date'] = pd.to_datetime(df_stock['Date'])
df_sent['date'] = pd.to_datetime(df_sent['date'])

# Step 2: Standardize column names
df_sent.rename(columns={'date': 'Date', 'ticker': 'Ticker'}, inplace=True)

# Step 3: Merge on 'Date' and 'Ticker'
df_merged = pd.merge(df_stock, df_sent, on=['Date', 'Ticker'], how='left')


In [57]:
df_merged.set_index('Date')
#df_merged = df_merged.drop(columns='title')
df_merged.head()

Date Ticker        Open        High         Low       Close  \
0 2024-03-11      A  146.668570  147.937054  144.805478  145.964951   
1 2024-03-11    ABT  117.968327  118.446324  116.690412  117.246452   
2 2024-03-11    ACN  366.420406  367.575311  359.265824  365.285065   
3 2024-03-11    ADM   52.694976   53.533242   52.675705   52.906952   
4 2024-03-11    AEE   70.142268   70.919704   70.113475   70.689354   

      Volume  sentiment  
0  1397600.0        NaN  
1  3886000.0        NaN  
2  2220000.0        NaN  
3  5209600.0        NaN  
4  1578300.0        NaN

In [58]:
df_merged.to_csv('stocks_signal_ahmedrachid_FinancialBERT.csv')